In [13]:
import requests
import json
import pandas as pd
from pprint import pprint
from census import Census
from us import states

#Census API Key
from config import api_key
c = Census(api_key, year=2017)
c2= Census(api_key, year=2011)

In [14]:
states = ["Alabama", "Alaska", "Arizona","Arkansas","California","Colorado","Connecticut","Delaware","District of Columbia","Florida","Georgia",
          "Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts",
          "Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey",
          "New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island",
          "South Carolina","South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin",
          "Wyoming"
]

df = pd.DataFrame(data={"States": states})
df["Median Income"] = ""
df["Home Value"] = ""
df["Bachelors Degree"] = ""
df["Population"] = ""
df["Divorce"] = ""

df.head()


,States,Median Income,Home Value,Bachelors Degree,Population,Divorce
0,Alabama,,,,,
1,Alaska,,,,,
2,Arizona,,,,,
3,Arkansas,,,,,
4,California,,,,,


In [26]:
# Pull US Census Data for each State
census_data = c.acs1.get(("NAME", "B01003_001E", "B15003_022E","B15003_023E","B15003_024E","B15003_025E",
                          "B23025_002E","B23025_005E", "B21003_001E", "B15003_017E","B15003_018E","B15003_019E",
                         "B15003_020E","B15003_021E"), {'for': 'state:*'})
census_2017 = pd.DataFrame(census_data)


# B15003_022E = Bachelors Degree
# B15003_023E = Masters
# B15003_024E = Professional
# B15003_025E = Doctor
# B19013_001E = Household income
# B01003_001E = Population
# B01002_001E = Median Age
# B19301_001E = Per Capita Income
# B15003_023E = Masters Degree
# B23025_002E = Labor Force
# B23025_004E = Employed
# B23025_005E = Unemployed
# B25077_001E = Median Home Value
# B08136_003E = Commute Time (in minutes)


census_2017 = census_2017.rename(columns={"B01003_001E": "Population",
                                      "B15003_022E": "Bachelors",
                                      "B15003_023E": "Masters",
                                      "B15003_024E": "Professional",
                                      "B15003_025E": "Doctor",
                                      "B23025_002E": "Labor Force",
                                      "B23025_005E": "Unemployed",
                                      "B21003_001E": "Pop25",
                                      "B15003_017E": "HS",
                                      "B15003_018E": "GED",
                                      "B15003_019E": "Under 1",
                                      "B15003_020E": "Over 1",
                                      "B15003_021E": "Associates",
                                      "NAME": "State", "state": "Code"})

# Add dataframe columns for bachelors and above as well as HS diploma and above (including GED)
census_2017["Bachelor+"] = census_2017["Bachelors"]+census_2017["Masters"]+census_2017["Professional"]+census_2017["Doctor"]
census_2017["HS+"]= census_2017["HS"]+census_2017["GED"]+census_2017["Bachelor+"]+census_2017["Under 1"]+census_2017["Over 1"]+census_2017["Associates"]

census_2017 = census_2017[["State", "Population","Pop25", "HS","GED","HS+","Bachelors", "Bachelor+","Labor Force", "Unemployed"]]

census_2017["Bachelors Rate"] = round((census_2017["Bachelor+"]/census_2017["Pop25"])*100,2)
census_2017["HS Diploma Rate"] = round(((census_2017["HS+"] + census_2017["GED"])/census_2017["Pop25"])*100,2)
census_2017["Unemployment Rate"] = round((census_2017["Unemployed"]/census_2017["Labor Force"])*100,2)

# Final Dataframe
census_2017 = census_2017[["State", "Population","Bachelors Rate", "HS Diploma Rate","Unemployment Rate"]]
# census_2017.to_csv("output.csv")



census_2017

,State,Population,Bachelors Rate,HS Diploma Rate,Unemployment Rate
0,Alabama,4874747.0,25.60,92.22,5.81
1,Alaska,739795.0,29.35,99.27,7.25
2,Arizona,7016270.0,29.43,91.37,5.77
3,Arkansas,3004279.0,23.46,93.16,5.55
4,California,39536653.0,33.73,85.81,5.87
5,Colorado,5607154.0,41.44,96.12,4.16
6,Connecticut,3588184.0,38.76,93.70,6.08
7,Delaware,961939.0,31.59,94.17,5.24
8,District of Columbia,693972.0,57.60,93.24,6.54
9,Florida,20984400.0,29.72,92.71,5.47


In [27]:
# Build exact same dataframe for 2011 data
census_data2 = c2.acs1.get(("NAME", "B01003_001E", "B15003_022E","B15003_023E","B15003_024E","B15003_025E",
                          "B23025_002E","B23025_005E", "B21003_001E", "B15003_017E","B15003_018E","B15003_019E",
                         "B15003_020E","B15003_021E"), {'for': 'state:*'})
census_2011 = pd.DataFrame(census_data2)

census_2011 = census_2011.rename(columns={"B01003_001E": "Population",
                                      "B15003_022E": "Bachelors",
                                      "B15003_023E": "Masters",
                                      "B15003_024E": "Professional",
                                      "B15003_025E": "Doctor",
                                      "B23025_002E": "Labor Force",
                                      "B23025_005E": "Unemployed",
                                      "B21003_001E": "Pop25",
                                      "B15003_017E": "HS",
                                      "B15003_018E": "GED",
                                      "B15003_019E": "Under 1",
                                      "B15003_020E": "Over 1",
                                      "B15003_021E": "Associates",
                                      "NAME": "State", "state": "Code"})

census_2011["Bachelor+"] = census_2011["Bachelors"]+census_2011["Masters"]+census_2011["Professional"]+census_2011["Doctor"]
census_2011["HS+"]= census_2011["HS"]+census_2011["GED"]+census_2011["Bachelor+"]+census_2011["Under 1"]+census_2011["Over 1"]+census_2011["Associates"]

census_2011 = census_2011[["State", "Population","Pop25", "HS","GED","HS+","Bachelors", "Bachelor+","Labor Force", "Unemployed"]]

census_2011["Bachelors Rate"] = round((census_2011["Bachelor+"]/census_2011["Pop25"])*100,2)
census_2011["HS Diploma Rate"] = round(((census_2011["HS+"] + census_2011["GED"])/census_2011["Pop25"])*100,2)
census_2011["Unemployment Rate"] = round((census_2011["Unemployed"]/census_2011["Labor Force"])*100,2)

census_2011 = census_2011[["State", "Population","Bachelors Rate", "HS Diploma Rate","Unemployment Rate"]]

census_2011

,State,Population,Bachelors Rate,HS Diploma Rate,Unemployment Rate
0,Alabama,4802740.0,22.33,88.27,11.50
1,Alaska,722718.0,26.96,99.11,8.46
2,Arizona,6482505.0,26.64,89.86,11.09
3,Arkansas,2937979.0,20.35,89.93,9.31
4,California,37691912.0,30.39,83.53,12.25
5,Colorado,5116796.0,36.88,94.53,9.19
6,Connecticut,3580709.0,36.23,92.47,10.57
7,Delaware,907135.0,28.95,90.49,8.88
8,District of Columbia,617996.0,52.61,90.06,11.31
9,Florida,19057542.0,25.91,90.30,12.34


In [44]:
# Merge 2 dataframes
census_merged = pd.merge(census_2011, census_2017, how="inner",on="State",suffixes=("_2011", "_2017"))

# Reorder Columns for Readability
census_merged = census_merged[["State", "Population_2011", "Population_2017", "Bachelors Rate_2011", "Bachelors Rate_2017","Unemployment Rate_2011", "Unemployment Rate_2017"]]

# Calculate percent change between years for each metric
census_merged["Population Change"]=round((((census_merged["Population_2017"]-census_merged["Population_2011"])/census_merged["Population_2011"])*100),2)
census_merged["Education Change"]=round((((census_merged["Bachelors Rate_2017"]-census_merged["Bachelors Rate_2011"])/census_merged["Bachelors Rate_2011"])*100),2)
census_merged["Unemployment Change"]=round((((census_merged["Unemployment Rate_2017"]-census_merged["Unemployment Rate_2011"])/census_merged["Unemployment Rate_2011"])*100),2)

# Reorder Columns for Readability
census_merged=census_merged[["State","Population Change", "Education Change", "Unemployment Change"]]
census_merged

# rank order each metric
census_merged["Population Score"]= census_merged["Population Change"].rank()
census_merged["Education Score"]= census_merged["Education Change"].rank()
census_merged["Unemployment Score"]=census_merged["Unemployment Change"].rank(ascending=False)

# Reorder Columns for Readability
census_merged=census_merged[["State","Population Change", "Population Score", "Education Change","Education Score", "Unemployment Change", "Unemployment Score"]]
census_merged["Composite Score"]=census_merged["Population Score"]+census_merged["Education Score"]+census_merged["Unemployment Score"]
census_merged

,State,Population Change,Population Score,Education Change,Education Score,Unemployment Change,Unemployment Score,Composite Score
0,Alabama,1.50,15.0,14.64,41.0,-49.48,30.0,86.0
1,Alaska,2.36,24.0,8.86,9.0,-14.30,4.0,37.0
2,Arizona,8.23,43.0,10.47,17.0,-47.97,29.0,89.0
3,Arkansas,2.26,22.0,15.28,45.0,-40.39,12.0,79.0
4,California,4.89,35.0,10.99,22.0,-52.08,37.0,94.0
5,Colorado,9.58,46.0,12.36,31.0,-54.73,44.0,121.0
6,Connecticut,0.21,6.0,6.98,4.0,-42.48,19.0,29.0
7,Delaware,6.04,38.0,9.12,10.0,-40.99,14.0,62.0
8,District of Columbia,12.29,52.0,9.48,13.0,-42.18,18.0,83.0
9,Florida,10.11,49.0,14.70,42.0,-55.67,48.0,139.0
